# Patrick de Carvalho Tavares Rezende Ferreira - RA: 175480 - EFC2

Repositório: https://github.com/patrickctrf/EA072-Inteligencia-Artificial-IA/tree/master/EFC1

In [0]:
%cd /content/drive/My\ Drive/PODE\ APAGAR/EA072-EF1

/content/drive/My Drive/PODE APAGAR/EA072-EF1


## Questão 1

Inicialmente, executou-se 5 vezes o código sugerido inicial a fim de verificar o desempenho da proposta. Seu desempenho foi de:

* Loss: 0.0733; Acurácia: 0.9775.

Utilizando-se o método de tentativa e erro, foi criado um script que verificava o desempenho da rede para diferentes parâmetros alterados como dropout (0.1 a 0.6), número de camadas (1 a 4 intermediárias), épocas de treinamento (4 a 8) e número de neurônios por camada (256 a 512). O script executava esta mudança de parâmetros dentro de loops "for" para executar todas as combinações possíveis e tirava também a média das múltiplas execuções com mesmos parâmetros, a fim de se obter uma média de desempenho mais confiável. Os resultados desta varredura eram salvos ao final das execuções em um arquivo "listas.txt", permitindo ao usuários verificar qual a configuração obteve melhor desempenho.
Foram utilizadas 4 threads - para varredura de redes de 1 a 4 camadas - durante o treinamento, a fim de promover paralelismo e diminuir o tempo requerido, que chegava a dezenas de horas.

Para a proposta final deste modelo, os parâmetros que resultaram no melhor desempenho durante a varredura foram:

* Camadas: 2; Neurônios por camada: 512; Dropout: 0.2; Épocas: 8.

O desempenho médio obtido foi de:

* Loss: 0.0693; Acurácia: 0.9828.

Ambas as soluções consumiram um tempo de execução da ordem de poucos minutos e a diferença de desempenho foi cerca de 0,5% em ganho.

Os arquivos utilizados foram (no diretório q1):

Proposta Inicial: q1Inicial.py
Script de Varredura de parâmetros: q1.py
Proposta Final: q1Final.py

In [0]:
# q1Inicial.py

import tensorflow as tf
import os
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
model = tf.keras.models.Sequential([
 tf.keras.layers.Flatten(),
 tf.keras.layers.Dense(512, activation=tf.nn.relu),
 tf.keras.layers.Dropout(0.5),
 tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
 loss='sparse_categorical_crossentropy',
 metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)
model_json = model.to_json()
json_file = open("model_MLP.json", "w")
json_file.write(model_json)
json_file.close()
model.save_weights("model_MLP.h5")
print("Model saved to disk")
os.getcwd()

Epoch 1/5
60000/60000 [==============================] - 6s 103us/sample - loss: 0.2698 - acc: 0.9197
Epoch 2/5
60000/60000 [==============================] - 6s 101us/sample - loss: 0.1385 - acc: 0.9579
Epoch 3/5
60000/60000 [==============================] - 6s 103us/sample - loss: 0.1080 - acc: 0.9668
Epoch 4/5
60000/60000 [==============================] - 6s 101us/sample - loss: 0.0928 - acc: 0.9710
Epoch 5/5
10000/10000 [==============================] - 1s 68us/sample - loss: 0.0640 - acc: 0.9797
Model saved to disk


'/content/drive/My Drive/PODE APAGAR/EA072-EF1'

In [0]:
import tensorflow as tf
import os
import threading

myMutex = threading.Lock()
value = "teste"

numeroDeNeuronios = []
numeroDeEpocas = []
numeroDeCamadas = []
numeroDeDropout = []
taxaDeAcertos = []

# Vamos colocar uma thread para treinar cada rede com um numero especifico de camadas.
def thread1Camadas(camadas):
	
	# Para tirar a media das iteracoes, somaremos todas aqui e dividiremos pelo total.
	somaDasEficienciasDeCadaIteracao = 0
	
	# Os valores que utilizaremos para dropout variarao de 10% a 90% (instrucao abaixo).
	valoresDropout = range(10, 60, 10)# Variaremos de 10% em 10%.
	valoresDropout = [i/100 for i in valoresDropout]# Converte de porcentagem para escala de 0 a 1.
	
	# Testando resultados com diferentes quantidades de epocas.
	for epocas in [8, 4]:
	
		# Testando resultados com diferentes quantidades de neuronios.
		for neuronios in [256, 512]:
		
			# So para indicar em que passo da execucao estamos.
			print("\n\nepocas: " + str(epocas) + "\nCAMADAS" + str(camadas) + ": " + str(neuronios) + "\n\n")
	
			# Este loop fica responsável por treinar com diferentes taxas de dropout.
			# "i" eh o valor a cada iteracao.
			for taxaDropout in valoresDropout:
			
				# Repetimos o treinamento algumas vezes para tirar uma media da eficiencia
				for iteracaoMedia in range(1,4):
					mnist = tf.keras.datasets.mnist
					(x_train, y_train),(x_test, y_test) = mnist.load_data()
					x_train, x_test = x_train / 255.0, x_test / 255.0
					model = tf.keras.models.Sequential([
					 tf.keras.layers.Flatten(),
					 tf.keras.layers.Dense(neuronios, activation=tf.nn.relu),
					 tf.keras.layers.Dropout(taxaDropout),# Diferentes valores de dropout.
					 tf.keras.layers.Dense(10, activation=tf.nn.softmax)
					])
					model.compile(optimizer='adam',
					 loss='sparse_categorical_crossentropy',
					 metrics=['accuracy'])
					model.fit(x_train, y_train, epochs=epocas)
					value = model.evaluate(x_test, y_test)
					model_json = model.to_json()
					json_file = open("model_MLP1.json", "w")
					json_file.write(model_json)
					json_file.close()
					model.save_weights("model_MLP1.h5")
					print("Model saved to disk")
					os.getcwd()
					
					somaDasEficienciasDeCadaIteracao = value[1] + somaDasEficienciasDeCadaIteracao
					
				
				
				myMutex.acquire()
				numeroDeNeuronios.append(neuronios)
				numeroDeEpocas.append(epocas)
				numeroDeCamadas.append(camadas)
				numeroDeDropout.append(taxaDropout)
				taxaDeAcertos.append(somaDasEficienciasDeCadaIteracao/iteracaoMedia)
				myMutex.release()
				
				# Reiniciamos a soma.
				somaDasEficienciasDeCadaIteracao = 0
				
# Vamos colocar uma thread para treinar cada rede com um numero especifico de camadas.
def thread2Camadas(camadas):
	
	# Para tirar a media das iteracoes, somaremos todas aqui e dividiremos pelo total.
	somaDasEficienciasDeCadaIteracao = 0
	
	# Os valores que utilizaremos para dropout variarao de 10% a 90% (instrucao abaixo).
	valoresDropout = range(10, 60, 10)# Variaremos de 10% em 10%.
	valoresDropout = [i/100 for i in valoresDropout]# Converte de porcentagem para escala de 0 a 1.
	
	# Testando resultados com diferentes quantidades de epocas.
	for epocas in [8, 4]:
	
		# Testando resultados com diferentes quantidades de neuronios.
		for neuronios in [256, 512]:
		
			# So para indicar em que passo da execucao estamos.
			print("\n\nepocas: " + str(epocas) + "\nCAMADAS" + str(camadas) + ": " + str(neuronios) + "\n\n")
	
			# Este loop fica responsável por treinar com diferentes taxas de dropout.
			# "i" eh o valor a cada iteracao.
			for taxaDropout in valoresDropout:
			
				# Repetimos o treinamento algumas vezes para tirar uma media da eficiencia
				for iteracaoMedia in range(1,4):
					mnist = tf.keras.datasets.mnist
					(x_train, y_train),(x_test, y_test) = mnist.load_data()
					x_train, x_test = x_train / 255.0, x_test / 255.0
					model = tf.keras.models.Sequential([
					 tf.keras.layers.Flatten(),
					 tf.keras.layers.Dense(neuronios, activation=tf.nn.relu),
					 tf.keras.layers.Dropout(taxaDropout),# Diferentes valores de dropout.
					 tf.keras.layers.Dense(neuronios, activation=tf.nn.relu),
					 tf.keras.layers.Dropout(taxaDropout),# Diferentes valores de dropout.
					 tf.keras.layers.Dense(10, activation=tf.nn.softmax)
					])
					model.compile(optimizer='adam',
					 loss='sparse_categorical_crossentropy',
					 metrics=['accuracy'])
					model.fit(x_train, y_train, epochs=epocas)
					value = model.evaluate(x_test, y_test)
					model_json = model.to_json()
					json_file = open("model_MLP2.json", "w")
					json_file.write(model_json)
					json_file.close()
					model.save_weights("model_MLP2.h5")
					print("Model saved to disk")
					os.getcwd()
					
					somaDasEficienciasDeCadaIteracao = value[1] + somaDasEficienciasDeCadaIteracao
					
				
				
				myMutex.acquire()
				numeroDeNeuronios.append(neuronios)
				numeroDeEpocas.append(epocas)
				numeroDeCamadas.append(camadas)
				numeroDeDropout.append(taxaDropout)
				taxaDeAcertos.append(somaDasEficienciasDeCadaIteracao/iteracaoMedia)
				myMutex.release()
				
				# Reiniciamos a soma.
				somaDasEficienciasDeCadaIteracao = 0
				
# Vamos colocar uma thread para treinar cada rede com um numero especifico de camadas.
def thread3Camadas(camadas):
	
	# Para tirar a media das iteracoes, somaremos todas aqui e dividiremos pelo total.
	somaDasEficienciasDeCadaIteracao = 0
	
	# Os valores que utilizaremos para dropout variarao de 10% a 90% (instrucao abaixo).
	valoresDropout = range(10, 60, 10)# Variaremos de 10% em 10%.
	valoresDropout = [i/100 for i in valoresDropout]# Converte de porcentagem para escala de 0 a 1.
	
	# Testando resultados com diferentes quantidades de epocas.
	for epocas in [8, 4]:
	
		# Testando resultados com diferentes quantidades de neuronios.
		for neuronios in [256, 512]:
		
			# So para indicar em que passo da execucao estamos.
			print("\n\nepocas: " + str(epocas) + "\nCAMADAS" + str(camadas) + ": " + str(neuronios) + "\n\n")
	
			# Este loop fica responsável por treinar com diferentes taxas de dropout.
			# "i" eh o valor a cada iteracao.
			for taxaDropout in valoresDropout:
			
				# Repetimos o treinamento algumas vezes para tirar uma media da eficiencia
				for iteracaoMedia in range(1,4):
					mnist = tf.keras.datasets.mnist
					(x_train, y_train),(x_test, y_test) = mnist.load_data()
					x_train, x_test = x_train / 255.0, x_test / 255.0
					model = tf.keras.models.Sequential([
					 tf.keras.layers.Flatten(),
					 tf.keras.layers.Dense(neuronios, activation=tf.nn.relu),
					 tf.keras.layers.Dropout(taxaDropout),# Diferentes valores de dropout.
					 tf.keras.layers.Dense(neuronios, activation=tf.nn.relu),
					 tf.keras.layers.Dropout(taxaDropout),# Diferentes valores de dropout.
					 tf.keras.layers.Dense(neuronios, activation=tf.nn.relu),
					 tf.keras.layers.Dropout(taxaDropout),# Diferentes valores de dropout.
					 tf.keras.layers.Dense(10, activation=tf.nn.softmax)
					])
					model.compile(optimizer='adam',
					 loss='sparse_categorical_crossentropy',
					 metrics=['accuracy'])
					model.fit(x_train, y_train, epochs=epocas)
					value = model.evaluate(x_test, y_test)
					model_json = model.to_json()
					json_file = open("model_MLP3.json", "w")
					json_file.write(model_json)
					json_file.close()
					model.save_weights("model_MLP3.h5")
					print("Model saved to disk")
					os.getcwd()
					
					somaDasEficienciasDeCadaIteracao = value[1] + somaDasEficienciasDeCadaIteracao
					
				
				
				myMutex.acquire()
				numeroDeNeuronios.append(neuronios)
				numeroDeEpocas.append(epocas)
				numeroDeCamadas.append(camadas)
				numeroDeDropout.append(taxaDropout)
				taxaDeAcertos.append(somaDasEficienciasDeCadaIteracao/iteracaoMedia)
				myMutex.release()
				
				# Reiniciamos a soma.
				somaDasEficienciasDeCadaIteracao = 0
				
# Vamos colocar uma thread para treinar cada rede com um numero especifico de camadas.
def thread4Camadas(camadas):
	
	# Para tirar a media das iteracoes, somaremos todas aqui e dividiremos pelo total.
	somaDasEficienciasDeCadaIteracao = 0
	
	# Os valores que utilizaremos para dropout variarao de 10% a 90% (instrucao abaixo).
	valoresDropout = range(10, 60, 10)# Variaremos de 10% em 10%.
	valoresDropout = [i/100 for i in valoresDropout]# Converte de porcentagem para escala de 0 a 1.
	
	# Testando resultados com diferentes quantidades de epocas.
	for epocas in [8, 4]:
	
		# Testando resultados com diferentes quantidades de neuronios.
		for neuronios in [256, 512]:
		
			# So para indicar em que passo da execucao estamos.
			print("\n\nepocas: " + str(epocas) + "\nCAMADAS" + str(camadas) + ": " + str(neuronios) + "\n\n")
	
			# Este loop fica responsável por treinar com diferentes taxas de dropout.
			# "i" eh o valor a cada iteracao.
			for taxaDropout in valoresDropout:
			
				# Repetimos o treinamento algumas vezes para tirar uma media da eficiencia
				for iteracaoMedia in range(1,4):
					mnist = tf.keras.datasets.mnist
					(x_train, y_train),(x_test, y_test) = mnist.load_data()
					x_train, x_test = x_train / 255.0, x_test / 255.0
					model = tf.keras.models.Sequential([
					 tf.keras.layers.Flatten(),
					 tf.keras.layers.Dense(neuronios, activation=tf.nn.relu),
					 tf.keras.layers.Dropout(taxaDropout),# Diferentes valores de dropout.
					 tf.keras.layers.Dense(neuronios, activation=tf.nn.relu),
					 tf.keras.layers.Dropout(taxaDropout),# Diferentes valores de dropout.
					 tf.keras.layers.Dense(neuronios, activation=tf.nn.relu),
					 tf.keras.layers.Dropout(taxaDropout),# Diferentes valores de dropout.
					 tf.keras.layers.Dense(neuronios, activation=tf.nn.relu),
					 tf.keras.layers.Dropout(taxaDropout),# Diferentes valores de dropout.
					 tf.keras.layers.Dense(10, activation=tf.nn.softmax)
					])
					model.compile(optimizer='adam',
					 loss='sparse_categorical_crossentropy',
					 metrics=['accuracy'])
					model.fit(x_train, y_train, epochs=epocas)
					value = model.evaluate(x_test, y_test)
					model_json = model.to_json()
					json_file = open("model_MLP4.json", "w")
					json_file.write(model_json)
					json_file.close()
					model.save_weights("model_MLP4.h5")
					print("Model saved to disk")
					os.getcwd()
					
					somaDasEficienciasDeCadaIteracao = value[1] + somaDasEficienciasDeCadaIteracao
					
				
				
				myMutex.acquire()
				numeroDeNeuronios.append(neuronios)
				numeroDeEpocas.append(epocas)
				numeroDeCamadas.append(camadas)
				numeroDeDropout.append(taxaDropout)
				taxaDeAcertos.append(somaDasEficienciasDeCadaIteracao/iteracaoMedia)
				myMutex.release()
				
				# Reiniciamos a soma.
				somaDasEficienciasDeCadaIteracao = 0
				
				
	
if __name__ == '__main__':

	
	camadas1 = threading.Thread(target=thread1Camadas,args=(1,))
	camadas2 = threading.Thread(target=thread2Camadas,args=(2,))
	camadas3 = threading.Thread(target=thread3Camadas,args=(3,))
	camadas4 = threading.Thread(target=thread4Camadas,args=(4,))
	
	camadas1.start()
	camadas2.start()
	camadas3.start()
	camadas4.start()
	
	try:
		camadas4.join(); 
	except:
		pass;
		
	try:
		camadas3.join(); 
	except:
		pass;
		
	try:
		camadas2.join(); 
	except:
		pass;
		
	try:
		camadas1.join(); 
	except:
		pass;
		
	listasFile = open("listasFFULLYCONNECTED.txt", "w")
	listasFile.write(str(numeroDeNeuronios) + "\n")
	listasFile.write(str(numeroDeEpocas) + "\n")
	listasFile.write(str(numeroDeCamadas) + "\n")
	listasFile.write(str(numeroDeDropout) + "\n")
	listasFile.write(str(taxaDeAcertos) + "\n")
	listasFile.close()
		
		
		



epocas: 8
CAMADAS1: 256




epocas: 8
CAMADAS2: 256




epocas: 8
CAMADAS3: 256




epocas: 8
CAMADAS4: 256


Epoch 1/8
 6112/60000 [==>...........................] - ETA: 12s - loss: 0.6049 - acc: 0.8253Epoch 1/8
Epoch 1/8 1760/60000 [..............................] - ETA: 32s - loss: 1.0388 - acc: 0.7000
60000/60000 [==============================] - 23s 379us/sample - loss: 0.2369 - acc: 0.9305
Epoch 2/8
60000/60000 [==============================] - 26s 434us/sample - loss: 0.2229 - acc: 0.9330
Epoch 2/8
 7616/60000 [==>...........................]60000/60000 [==============================] - ETA: 23s - loss: 0.1076 - acc: 0.9645 - 29s 476us/sample - loss: 0.2287 - acc: 0.9308
Epoch 2/8
60000/60000 [==============================] 3328/60000 [>.............................] - 30s 492us/sample - loss: 0.2444 - acc: 0.9270
 - ETA: 26s - loss: 0.1149 - acc: 0.9645Epoch 2/8
60000/60000 [==============

In [0]:
# q1Final.py

import tensorflow as tf
import os
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
model = tf.keras.models.Sequential([
 tf.keras.layers.Flatten(),
 tf.keras.layers.Dense(512, activation=tf.nn.relu),
 tf.keras.layers.Dropout(0.2),
 tf.keras.layers.Dense(512, activation=tf.nn.relu),
 tf.keras.layers.Dropout(0.2),
 tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
 loss='sparse_categorical_crossentropy',
 metrics=['accuracy'])
model.fit(x_train, y_train, epochs=8)
model.evaluate(x_test, y_test)
model_json = model.to_json()
json_file = open("model_MLP.json", "w")
json_file.write(model_json)
json_file.close()
model.save_weights("model_MLP.h5")
print("Model saved to disk")
os.getcwd()

Epoch 1/8
60000/60000 [==============================] - 8s 135us/sample - loss: 0.2130 - acc: 0.9347
Epoch 2/8
60000/60000 [==============================] - 8s 128us/sample - loss: 0.1075 - acc: 0.9674
Epoch 3/8
60000/60000 [==============================] - 8s 129us/sample - loss: 0.0810 - acc: 0.9745
Epoch 4/8
60000/60000 [==============================] - 8s 125us/sample - loss: 0.0658 - acc: 0.9791
Epoch 5/8
60000/60000 [==============================] - 8s 127us/sample - loss: 0.0563 - acc: 0.9820
Epoch 6/8
60000/60000 [==============================] - 8s 127us/sample - loss: 0.0505 - acc: 0.9839
Epoch 7/8
60000/60000 [==============================] - 8s 127us/sample - loss: 0.0472 - acc: 0.9856
Epoch 8/8
10000/10000 [==============================] - 1s 77us/sample - loss: 0.0721 - acc: 0.9828
Model saved to disk


'/content/drive/My Drive/PODE APAGAR/EA072-EF1'

## Questão 2

Inicialmente, executou-se 5 vezes o código sugerido inicial a fim de verificar o desempenho da proposta. Seu desempenho foi de:

* Loss: 0.0260; Acurácia: 0.9909.

Utilizando-se o método de tentativa e erro, foi criado um script que verificava o desempenho da rede para diferentes parâmetros alterados como dropout (0.1 a 0.6), número de filtros (32 a 64), épocas de treinamento (2 a 6) e formato dos kernel utilizados (2x2 ou 3x3). O script executava esta mudança de parâmetros dentro de loops "for" para executar todas as combinações possíveis e tirava também a média das múltiplas execuções com mesmos parâmetros, a fim de se obter uma média de desempenho mais confiável. Os resultados desta varredura eram salvos ao final das execuções em um arquivo "listas.txt", permitindo ao usuários verificar qual a configuração obteve melhor desempenho.
Foram utilizadas 4 threads - para varredura de redes de 1 a 4 camadas - durante o treinamento, a fim de promover paralelismo e diminuir o tempo requerido, que era ainda maior que o demandado para a questão 1. Verificou-se que com 6 épocas de treinamento o resultdo era levemente melhorado, mas não siginficativamente. A variação das demais grandezas fazia o desempenho diminuir nos testes. Então, após a varredura, foi realizado mais uma tentativa de treinamento com adição de uma camada convolucional e dropout de 0.3, o que elevou os resultados e nos trouxe à proposta final de código para esta questão.

Para a proposta final deste modelo, os parâmetros alterados que resultaram no melhor desempenho durante a varredura foram:

* Adição de uma camada convolucional (seguida de uma max pool) após a saída da primeira layer de max pool; 6 épocas de treinamento. Os demais parâmetros foram mantidos por não apresentar vantagem média significativa.

O desempenho médio obtido foi de:

* Loss: 0.0190; Acurácia: 0.9931.

Ambas as soluções consumiram um tempo de execução da ordem de poucos minutos e a diferença de desempenho foi cerca de 0,22% em ganho.

Os arquivos utilizados foram (no diretório q2):

Proposta Inicial: q2Inicial.py
Script de Varredura de parâmetros: q2.py
Proposta Final: q2Final.py

## Comparação entre ELM, MLP e CNN

Desempenho:
* ELM: 91,09%
* MLP: 98,28%
* CNN: 99,31%

Nota-se claramente que a CNN apresenta o melhor desempenho dentre as 3 melhores técnicas utilizadas. Porém, o processo de treinamento para otimização desta toma dezenas de horas, enquanto que a ELM requer apenas alguns minutos para ser ajustada e ficar cerca de 8% abaixo em desempenho. Logo, se houver recursos computacionais, a CNN é a melhor escolha para este tipo de problema.

In [0]:
# q2Inicial.py

import tensorflow as tf
import os
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
# reshape to be [samples][width][height][pixels]
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train, x_test = x_train / 255.0, x_test / 255.0
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
 activation='relu',
input_shape=(28, 28, 1)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam',
 loss='sparse_categorical_crossentropy',
 metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)
model_json = model.to_json()
json_file = open("model_CNN.json", "w")
json_file.write(model_json)
json_file.close()
model.save_weights("model_CNN.h5")
print("Model saved to disk")
os.getcwd()

Epoch 1/5
60000/60000 [==============================] - 16s 268us/sample - loss: 0.2004 - acc: 0.9403
Epoch 2/5
60000/60000 [==============================] - 15s 256us/sample - loss: 0.0851 - acc: 0.9741
Epoch 3/5
60000/60000 [==============================] - 15s 251us/sample - loss: 0.0628 - acc: 0.9809
Epoch 4/5
60000/60000 [==============================] - 15s 252us/sample - loss: 0.0532 - acc: 0.9837
Epoch 5/5
10000/10000 [==============================] - 1s 114us/sample - loss: 0.0294 - acc: 0.9906
Model saved to disk


'/content/drive/My Drive/PODE APAGAR/EA072-EF1'

In [0]:
# q2.py

import tensorflow as tf
import os
import threading

myMutex = threading.Lock()
value = "teste"

numeroDeNeuronios = []
numeroDeEpocas = []
numeroDeCamadas = []
numeroDeDropout = []
taxaDeAcertos = []

# Vamos colocar uma thread para treinar cada rede com um numero especifico de camadas.
def thread1Camadas(camadas):
	
	# Para tirar a media das iteracoes, somaremos todas aqui e dividiremos pelo total.
	somaDasEficienciasDeCadaIteracao = 0
	
	# Os valores que utilizaremos para dropout variarao de 10% a 90% (instrucao abaixo).
	valoresDropout = range(10, 40, 10)# Variaremos de 10% em 10%.
	valoresDropout = [i/100 for i in valoresDropout]# Converte de porcentagem para escala de 0 a 1.
	
	# Testando resultados com diferentes quantidades de epocas.
	for epocas in [2, 6]:
	
		# Testando resultados com diferentes quantidades de filtros.
		for filtros in [32, 64]:
		
			# So para indicar em que passo da execucao estamos.
			print("\n\nepocas: " + str(epocas) + "\nCAMADAS" + str(camadas) + ": " + str(filtros) + "\n\n")
	
			# Este loop fica responsável por treinar com diferentes taxas de dropout.
			# "i" eh o valor a cada iteracao.
			for taxaDropout in valoresDropout:
			
				# Repetimos o treinamento algumas vezes para tirar uma media da eficiencia
				for iteracaoMedia in range(1,3):
					mnist = tf.keras.datasets.mnist
					(x_train, y_train),(x_test, y_test) = mnist.load_data()
					# reshape to be [samples][width][height][pixels]
					x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
					x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
					x_train, x_test = x_train / 255.0, x_test / 255.0
					model = tf.keras.models.Sequential()
					model.add(tf.keras.layers.Conv2D(filtros, kernel_size=(3, 3),
					 activation='relu',
					input_shape=(28, 28, 1)))
					model.add(tf.keras.layers.Conv2D(filtros*2, (3, 3), activation='relu'))
					model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
					model.add(tf.keras.layers.Dropout(taxaDropout))
					model.add(tf.keras.layers.Flatten())
					model.add(tf.keras.layers.Dense(128, activation='relu'))
					model.add(tf.keras.layers.Dropout(taxaDropout))
					model.add(tf.keras.layers.Dense(10, activation='softmax'))
					model.compile(optimizer='adam',
					 loss='sparse_categorical_crossentropy',
					 metrics=['accuracy'])
					model.fit(x_train, y_train, epochs=epocas)
					value = model.evaluate(x_test, y_test)
					model_json = model.to_json()
					json_file = open("model_CNN1.json", "w")
					json_file.write(model_json)
					json_file.close()
					model.save_weights("model_CNN1.h5")
					print("Model saved to disk")
					os.getcwd()
					
					somaDasEficienciasDeCadaIteracao = value[1] + somaDasEficienciasDeCadaIteracao
					
				
				
				myMutex.acquire()
				numeroDeNeuronios.append(filtros)
				numeroDeEpocas.append(epocas)
				numeroDeCamadas.append(camadas)
				numeroDeDropout.append(taxaDropout)
				taxaDeAcertos.append(somaDasEficienciasDeCadaIteracao/iteracaoMedia)
				myMutex.release()
				
				# Reiniciamos a soma.
				somaDasEficienciasDeCadaIteracao = 0
				
# Vamos colocar uma thread para treinar cada rede com um numero especifico de camadas.
def thread2Camadas(camadas):
	
	# Para tirar a media das iteracoes, somaremos todas aqui e dividiremos pelo total.
	somaDasEficienciasDeCadaIteracao = 0
	
	# Os valores que utilizaremos para dropout variarao de 10% a 90% (instrucao abaixo).
	valoresDropout = range(10, 40, 10)# Variaremos de 10% em 10%.
	valoresDropout = [i/100 for i in valoresDropout]# Converte de porcentagem para escala de 0 a 1.
	
	# Testando resultados com diferentes quantidades de epocas.
	for epocas in [2, 6]:
	
		# Testando resultados com diferentes quantidades de filtros.
		for filtros in [32, 64]:
		
			# So para indicar em que passo da execucao estamos.
			print("\n\nepocas: " + str(epocas) + "\nCAMADAS" + str(camadas) + ": " + str(filtros) + "\n\n")
	
			# Este loop fica responsável por treinar com diferentes taxas de dropout.
			# "i" eh o valor a cada iteracao.
			for taxaDropout in valoresDropout:
			
				# Repetimos o treinamento algumas vezes para tirar uma media da eficiencia
				for iteracaoMedia in range(1,3):
					mnist = tf.keras.datasets.mnist
					(x_train, y_train),(x_test, y_test) = mnist.load_data()
					# reshape to be [samples][width][height][pixels]
					x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
					x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
					x_train, x_test = x_train / 255.0, x_test / 255.0
					model = tf.keras.models.Sequential()
					model.add(tf.keras.layers.Conv2D(filtros, kernel_size=(2, 2),
					 activation='relu',
					input_shape=(28, 28, 1)))
					model.add(tf.keras.layers.Conv2D(filtros*2, (3, 3), activation='relu'))
					model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
					model.add(tf.keras.layers.Conv2D(filtros*2, (3, 3), activation='relu'))
					model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
					model.add(tf.keras.layers.Dropout(taxaDropout))
					model.add(tf.keras.layers.Flatten())
					model.add(tf.keras.layers.Dense(128, activation='relu'))
					model.add(tf.keras.layers.Dropout(taxaDropout))
					model.add(tf.keras.layers.Dense(10, activation='softmax'))
					model.compile(optimizer='adam',
					 loss='sparse_categorical_crossentropy',
					 metrics=['accuracy'])
					model.fit(x_train, y_train, epochs=epocas)
					value = model.evaluate(x_test, y_test)
					model_json = model.to_json()
					json_file = open("model_CNN2.json", "w")
					json_file.write(model_json)
					json_file.close()
					model.save_weights("model_CNN2.h5")
					print("Model saved to disk")
					os.getcwd()
					
					somaDasEficienciasDeCadaIteracao = value[1] + somaDasEficienciasDeCadaIteracao
					
				
				
				
				myMutex.acquire()
				numeroDeNeuronios.append(filtros)
				numeroDeEpocas.append(epocas)
				numeroDeCamadas.append(camadas)
				numeroDeDropout.append(taxaDropout)
				taxaDeAcertos.append(somaDasEficienciasDeCadaIteracao/iteracaoMedia)
				myMutex.release()
				
				# Reiniciamos a soma.
				somaDasEficienciasDeCadaIteracao = 0
				
# Vamos colocar uma thread para treinar cada rede com um numero especifico de camadas.
def thread3Camadas(camadas):
	
	# Para tirar a media das iteracoes, somaremos todas aqui e dividiremos pelo total.
	somaDasEficienciasDeCadaIteracao = 0
	
	# Os valores que utilizaremos para dropout variarao de 10% a 90% (instrucao abaixo).
	valoresDropout = range(10, 40, 10)# Variaremos de 10% em 10%.
	valoresDropout = [i/100 for i in valoresDropout]# Converte de porcentagem para escala de 0 a 1.
	
	# Testando resultados com diferentes quantidades de epocas.
	for epocas in [2, 6]:
	
		# Testando resultados com diferentes quantidades de filtros.
		for filtros in [32, 64]:
		
			# So para indicar em que passo da execucao estamos.
			print("\n\nepocas: " + str(epocas) + "\nCAMADAS" + str(camadas) + ": " + str(filtros) + "\n\n")
	
			# Este loop fica responsável por treinar com diferentes taxas de dropout.
			# "i" eh o valor a cada iteracao.
			for taxaDropout in valoresDropout:
			
				# Repetimos o treinamento algumas vezes para tirar uma media da eficiencia
				for iteracaoMedia in range(1,3):
					mnist = tf.keras.datasets.mnist
					(x_train, y_train),(x_test, y_test) = mnist.load_data()
					# reshape to be [samples][width][height][pixels]
					x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
					x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
					x_train, x_test = x_train / 255.0, x_test / 255.0
					model = tf.keras.models.Sequential()
					model.add(tf.keras.layers.Conv2D(filtros, kernel_size=(3, 3),
					 activation='relu',
					input_shape=(28, 28, 1)))
					model.add(tf.keras.layers.Conv2D(filtros*2, (2, 2), activation='relu'))
					model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
					model.add(tf.keras.layers.Dropout(taxaDropout))
					model.add(tf.keras.layers.Flatten())
					model.add(tf.keras.layers.Dense(128, activation='relu'))
					model.add(tf.keras.layers.Dropout(taxaDropout))
					model.add(tf.keras.layers.Dense(10, activation='softmax'))
					model.compile(optimizer='adam',
					 loss='sparse_categorical_crossentropy',
					 metrics=['accuracy'])
					model.fit(x_train, y_train, epochs=epocas)
					value = model.evaluate(x_test, y_test)
					model_json = model.to_json()
					json_file = open("model_CNN3.json", "w")
					json_file.write(model_json)
					json_file.close()
					model.save_weights("model_CNN3.h5")
					print("Model saved to disk")
					os.getcwd()
					
					somaDasEficienciasDeCadaIteracao = value[1] + somaDasEficienciasDeCadaIteracao
					
				
				
				myMutex.acquire()
				numeroDeNeuronios.append(filtros)
				numeroDeEpocas.append(epocas)
				numeroDeCamadas.append(camadas)
				numeroDeDropout.append(taxaDropout)
				taxaDeAcertos.append(somaDasEficienciasDeCadaIteracao/iteracaoMedia)
				myMutex.release()
				
				# Reiniciamos a soma.
				somaDasEficienciasDeCadaIteracao = 0
				
# Vamos colocar uma thread para treinar cada rede com um numero especifico de camadas.
def thread4Camadas(camadas):
	
	# Para tirar a media das iteracoes, somaremos todas aqui e dividiremos pelo total.
	somaDasEficienciasDeCadaIteracao = 0
	
	# Os valores que utilizaremos para dropout variarao de 10% a 90% (instrucao abaixo).
	valoresDropout = range(10, 40, 10)# Variaremos de 10% em 10%.
	valoresDropout = [i/100 for i in valoresDropout]# Converte de porcentagem para escala de 0 a 1.
	
	# Testando resultados com diferentes quantidades de epocas.
	for epocas in [2, 6]:
	
		# Testando resultados com diferentes quantidades de filtros.
		for filtros in [32, 64]:
		
			# So para indicar em que passo da execucao estamos.
			print("\n\nepocas: " + str(epocas) + "\nCAMADAS" + str(camadas) + ": " + str(filtros) + "\n\n")
	
			# Este loop fica responsável por treinar com diferentes taxas de dropout.
			# "i" eh o valor a cada iteracao.
			for taxaDropout in valoresDropout:
			
				# Repetimos o treinamento algumas vezes para tirar uma media da eficiencia
				for iteracaoMedia in range(1,3):
					mnist = tf.keras.datasets.mnist
					(x_train, y_train),(x_test, y_test) = mnist.load_data()
					# reshape to be [samples][width][height][pixels]
					x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
					x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
					x_train, x_test = x_train / 255.0, x_test / 255.0
					model = tf.keras.models.Sequential()
					model.add(tf.keras.layers.Conv2D(filtros, kernel_size=(3, 3),
					 activation='relu',
					input_shape=(28, 28, 1)))
					model.add(tf.keras.layers.Conv2D(filtros*2, (3, 3), activation='relu'))
					model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3)))
					model.add(tf.keras.layers.Conv2D(filtros*2, (3, 3), activation='relu'))
					model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3)))
					model.add(tf.keras.layers.Dropout(taxaDropout))
					model.add(tf.keras.layers.Flatten())
					model.add(tf.keras.layers.Dense(128, activation='relu'))
					model.add(tf.keras.layers.Dropout(taxaDropout))
					model.add(tf.keras.layers.Dense(10, activation='softmax'))
					model.compile(optimizer='adam',
					 loss='sparse_categorical_crossentropy',
					 metrics=['accuracy'])
					model.fit(x_train, y_train, epochs=epocas)
					value = model.evaluate(x_test, y_test)
					model_json = model.to_json()
					json_file = open("model_CNN4.json", "w")
					json_file.write(model_json)
					json_file.close()
					model.save_weights("model_CNN4.h5")
					print("Model saved to disk")
					os.getcwd()
					
					somaDasEficienciasDeCadaIteracao = value[1] + somaDasEficienciasDeCadaIteracao
					
				
				
				
				myMutex.acquire()
				numeroDeNeuronios.append(filtros)
				numeroDeEpocas.append(epocas)
				numeroDeCamadas.append(camadas)
				numeroDeDropout.append(taxaDropout)
				taxaDeAcertos.append(somaDasEficienciasDeCadaIteracao/iteracaoMedia)
				myMutex.release()
				
				# Reiniciamos a soma.
				somaDasEficienciasDeCadaIteracao = 0
				
				
	
if __name__ == '__main__':

	
	camadas1 = threading.Thread(target=thread1Camadas,args=(1,))
	camadas2 = threading.Thread(target=thread2Camadas,args=(2,))
	camadas3 = threading.Thread(target=thread3Camadas,args=(3,))
	camadas4 = threading.Thread(target=thread4Camadas,args=(4,))
	
	camadas1.start()
	camadas2.start()
	camadas3.start()
	camadas4.start()
	
	try:
		camadas4.join(); 
	except:
		pass;
		
	try:
		camadas3.join(); 
	except:
		pass;
		
	try:
		camadas2.join(); 
	except:
		pass;
		
	try:
		camadas1.join(); 
	except:
		pass;
		
	listasFile = open("listasCONV.txt", "w")
	listasFile.write(str(numeroDeNeuronios) + "\n")
	listasFile.write(str(numeroDeEpocas) + "\n")
	listasFile.write(str(numeroDeCamadas) + "\n")
	listasFile.write(str(numeroDeDropout) + "\n")
	listasFile.write(str(taxaDeAcertos) + "\n")
	listasFile.close()
		
		
		



epocas: 2
CAMADAS1: 32



epocas: 2
CAMADAS2: 32





epocas: 2
CAMADAS3: 32




epocas: 2
CAMADAS4: 32


Epoch 1/2
60000/60000 [==============================] - 49s 822us/sample - loss: 0.1270 - acc: 0.9617
Epoch 2/2
60000/60000 [==============================] - 52s 870us/sample - loss: 0.1650 - acc: 0.9486
Epoch 2/2
60000/60000 [==============================] - 52s 860us/sample - loss: 0.1242 - acc: 0.9617
Epoch 2/2
10000/10000 [==============================] - 3s 327us/sample - loss: 0.0323 - acc: 0.9896
Model saved to disk
Epoch 1/2
60000/60000 [==============================] - 52s 872us/sample - loss: 0.1215 - acc: 0.9631
Epoch 2/2
 1856/60000 [..............................]60000/60000 [==============================] - 53s 888us/sample - loss: 0.1258 - acc: 0.9614
 - ETA: 50s - loss: 0.0392 - acc: 0.9898 - ETA: 38s - loss: 0.0414 - acc: 0.9876Epoch 2/2
60000/60000 [==============================]17280/60000 [=======>......................] - 50s 831us/sample - loss: 0.130

In [0]:
# q2Final.py
import tensorflow as tf
import os
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
# reshape to be [samples][width][height][pixels]
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train, x_test = x_train / 255.0, x_test / 255.0
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
 activation='relu',
input_shape=(28, 28, 1)))
model.add(tf.keras.layers.Conv2D(512, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Conv2D(512, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Conv2D(512, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam',
 loss='sparse_categorical_crossentropy',
 metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10)
model.evaluate(x_test, y_test)
model_json = model.to_json()
json_file = open("model_CNN.json", "w")
json_file.write(model_json)
json_file.close()
model.save_weights("model_CNN.h5")
print("Model saved to disk")
os.getcwd()

Epoch 1/10
60000/60000 [==============================] - 79s 1ms/sample - loss: 0.2189 - acc: 0.9315
Epoch 2/10
60000/60000 [==============================] - 78s 1ms/sample - loss: 0.0785 - acc: 0.9787
Epoch 3/10
60000/60000 [==============================] - 78s 1ms/sample - loss: 0.0625 - acc: 0.9830
Epoch 4/10
60000/60000 [==============================] - 78s 1ms/sample - loss: 0.0532 - acc: 0.9855
Epoch 5/10
60000/60000 [==============================] - 78s 1ms/sample - loss: 0.0457 - acc: 0.9871
Epoch 6/10
60000/60000 [==============================] - 78s 1ms/sample - loss: 0.0438 - acc: 0.9879
Epoch 7/10
60000/60000 [==============================] - 78s 1ms/sample - loss: 0.0376 - acc: 0.9899
Epoch 8/10
60000/60000 [==============================] - 78s 1ms/sample - loss: 0.0353 - acc: 0.9904
Epoch 9/10
60000/60000 [==============================] - 78s 1ms/sample - loss: 0.0340 - acc: 0.9907
Epoch 10/10
10000/10000 [==============================] - 4s 428us/sample - loss:

'/content'